In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from sklearnex import patch_sklearn 

patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [16]:
from os.path import expanduser, join
import pathlib

root = pathlib.Path().resolve()

In [17]:
from os import listdir

dir = listdir(root)
if 't.ex-Graph' in dir:
  root = join(root, 't.ex-Graph')

In [18]:
import sys
sys.path.insert(1, join(root, 'lib'))

import config
import data
import model
import export

In [19]:
datasets = [

  {
    'label': 'HTTP/S Graph (SLDs)',
    'data': data.read(join(root, 'data', 'graph-data-sld.csv'))
  }, 

  {
    'label': 'HTTP/S Graph (FQDN)',
    'data': data.read(join(root, 'data', 'graph-data-fqdn.csv'))
  }

]

In [20]:
for dataset in datasets:
  data.binary_classification_labels(config.binary_tracker, dataset.get('data'))

In [22]:
features = data.feature_vector(datasets[0].get('data'))

In [23]:
targets = [ 
  config.binary_tracker
]

In [24]:
models = dict()
for target in targets:
  models[target] = list()

In [25]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

models[config.binary_tracker] = [
    #LogisticRegression(solver='lbfgs', max_iter=10000, n_jobs=-1),
    KNeighborsClassifier(3, n_jobs=-1),
    #SVC(kernel="linear", C=0.025),
    #SVC(gamma=2, C=1),
    GaussianNB(),
    DecisionTreeClassifier(),
    RandomForestClassifier(max_depth=2, random_state=0, n_jobs=-1),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
    #QuadraticDiscriminantAnalysis(),
    #MLPClassifier(alpha=1, max_iter=1000)
]

In [26]:
%matplotlib agg

results = model.compute_results(
  datasets, 
  models, 
  features, 
  targets
)

In [42]:
export.classification_results(results, root)

In [43]:
"""%matplotlib inline"""

export.feature_importances(results, root)

In [44]:
export.cross_validation(results, root)